In [1]:
from IPython.display import Math
import math

In [2]:
# Number of series LEDs
N = 1
# Single LED forward voltage
VLED = 36 # Volts
# Single LED dynamic resistance
rLED = 0.02 # ohms
# Nominal input voltage
VIN = 14.4 # Volts nominal for 4S LiPo
# Input voltage range
VIN_MAX = 22.0 # Volts
VIN_MIN = 9 # Volts
# Switching frequency
fSW = 500e3 # Hz 
# Current sense voltage
VSNS = 100e-3 # Volts
# Average LED current
ILED = 1 # Amps
# Inductor current ripple
ΔiL_PP = 0.7 # Amps
# LED current ripple
ΔiLED_PP = 0.025 # Amps
# Peak current limit
ILIM = 2 # Amps
# Output OVLO characteristics
VTURN_OFF = 40 # Volts
VHYSO = 38 # Volts
# Intput UVLO characteristics
VTURN_ON = 3.5*3
VHYS = 3
# Thermal foldback characteristics
TBK = 85 # °C
TEND = 100 # °C
# Total start-up time
tTSU = 0.05 # Seconds
# nominal input voltage ripple (ΔvIN-PP)
ΔVIN_PP = 0.1 # Volts

In [3]:
# Operating Point
# Given the number of series LEDs (N), the forward voltage (VLED) and dynamic 
# resistance (rLED) for a single LED, solve for the nominal output voltage 
# (VO) and the nominal LED string dynamic resistance (rD):
VO = N*VLED
rD = N*rLED
print("V_out =", VO, "V")
print("rD =", rD, "ohms")

# Solve for the ideal nominal duty cycle (D)
# Boost Topology:
D = (VO-VIN)/VO
DMIN = (VO-VIN_MAX)/VO
DMAX = (VO-VIN_MIN)/VO
Dprime = 1-D
print("D =", D, "")
print("DMIN =", DMIN, "")
print("DMAX =", DMAX, "")
print("Dprime =", Dprime, "")

V_out = 36 V
rD = 0.02 ohms
D = 0.6000000000000001 
DMIN = 0.3888888888888889 
DMAX = 0.75 
Dprime = 0.3999999999999999 


In [4]:
# Switching Frequency
# Set the switching frequency (fSW) by solving for RT:
RT = (1+1.95e-8 * fSW) / (1.4e-10 * fSW)
print("RT =", RT, "Ohms")
print("round to 14.5 kOhm")

RT = 14424.999999999998 Ohms
round to 14.5 kOhm


In [5]:
# Average LED Current
# For all topologies, set the average LED current (ILED) knowing the desired 
# current sense voltage (VSNS) and solving for RSNS
RSNS = VSNS / ILED
print("RSNS =", RSNS, "Ohms")
# Setup the suggested signal current of 100 μA by assuming RCSH = 12.4 kΩ and solving for RHSP:
RCSH = 12400
RHSP = RHSN = (ILED * RCSH * RSNS)/1.24
print("RHSP = RHSN =", RHSP, "Ohms")

RSNS = 0.1 Ohms
RHSP = RHSN = 1000.0 Ohms


In [6]:
# Thermal Foldback
# For all topologies, set the thermal foldback breakpoint (TBK) by finding 
# corresponding RNTC-BK from manufacturer's datasheet and solving for RBIAS:
# Set Rref1 = Rref2
Rt25_85 = 0.081823 # lookup from datasheet
Rt25_100 = 0.048796 # lookup from datasheet 
RNTC_BK = 100000 * Rt25_85 
RNTC_END = 100000 * Rt25_100
print("RNTC_BK =", RNTC_BK, "Ohms")
print("RNTC_END =", RNTC_END, "Ohms")

RBIAS = RNTC_BK
RREF1 = RREF2 = 49900
ICSH  = 100e-6 # suggested signal current from TI
RGAIN = (RREF1/(RREF1+RREF2) - RNTC_END/( RNTC_END + RBIAS)) * 2.45 / ICSH 
print("RBIAS =", RBIAS, "Ohms")
print("RREF1 = RREF2 =", RREF1, "Ohms")
print("RGAIN =", RGAIN, "Ohms")

RNTC_BK = 8182.300000000001 Ohms
RNTC_END = 4879.6 Ohms
RBIAS = 8182.300000000001 Ohms
RREF1 = RREF2 = 49900 Ohms
RGAIN = 3097.4111729533984 Ohms


In [7]:
# Inductor Ripple Current
# Set the nominal inductor ripple current (ΔiL-PP) by solving for the appropriate inductor (L1):
# Boost and Buck-boost
L1 = (VIN * D) / (ΔiL_PP * fSW)
print("L1 =", L1, "Henrys") # try MSS1583-183MEB 18uH 20% 11.2 A
# To set the worst case inductor ripple current, use VIN-MAX and DMIN when solving for L1.
# The minimum allowable inductor RMS current rating (IL-RMS) can be calculated as:
# Boost and Buck-Boost
IL_RMS = ILED / Dprime * math.sqrt(1+1/12*(ΔiL_PP*Dprime/ILED)**2)
print("IL_RMS =", IL_RMS, "Amps")

L1 = 2.468571428571429e-05 Henrys
IL_RMS = 2.508153371174366 Amps


In [8]:
# LED Ripple Current
# Set the nominal LED ripple current (ΔiLED-PP), by solving for the output capacitance (CO):
# Boost and Buck-Boost
CO = ILED*D/(rD*ΔiL_PP*fSW)
print("CO =", CO, "Farads")
# To set the worst case LED ripple current, use DMAX when solving for CO. Remember, when PWM 
# dimming it is recommended to use a minimum of 40 μF of output capacitance to improve performance.
# The minimum allowable RMS output capacitor current rating (ICO-RMS) can be approximated:
# Boost and Buck-Boost
ICO_RMS = ILED*math.sqrt(DMAX/(1-DMAX))
print("ICO_RMS =", ICO_RMS, "Amps")

CO = 8.571428571428574e-05 Farads
ICO_RMS = 1.7320508075688772 Amps


In [9]:
# Peak Current Limit
# Set the peak current limit (ILIM) by solving for the transistor path sense resistor (RLIM):
RLIM = 0.245/ILIM
print("RLIM =", RLIM, "Ohms")

RLIM = 0.1225 Ohms


In [10]:
# Slope Compensation
# For all topologies, the preferred method to set slope compensation is to ensure any duty 
# cycle is attainable for the nominal VO and chosen L by solving for RSLP:
RSLP = 1.5e13*L1/(VO*RT*RLIM)
print("RSLP =", RSLP, "Ohms")
print("round to 407 kOhms")

RSLP = 5820.798237591647 Ohms
round to 407 kOhms


In [11]:
# Loop Compensation
# Using a simple first order peak current mode control model, neglecting any output 
# capacitor ESR dynamics, the necessary loop compensation can be determined.

# The uncompensated DC loop gain (TU0) is approximated: Boost
TU0 = Dprime*310/(ILED*RLIM)
# the pole (ωP1) is approximated: Boost
ωP1 = 2/(rD*CO)
# the RHP zero (ωZ1) is approximated: Boost
ωZ1 = rD*Dprime**2 / L1

# For all topologies, the primary method of compensation is to place a low frequency dominant pole (ωP2) which
# will ensure that there is ample phase margin at the crossover frequency. This is accomplished by placing a
# capacitor (CCMP) from the COMP pin to GND, which is calculated according to the lower value of the pole and the
# RHP zero of the system (shown as a minimizing function):
ωP2 = min(ωP1, ωZ1) / (5*TU0)
CCMP = 1/(ωP2*5e6)
# If analog dimming is used, CCMP should be approximately 4x larger to maintain stability as the LEDs are dimmed
# to zero.
# A high frequency compensation pole (ωP3) can be used to attenuate switching noise and provide better gain
# margin. Assuming RFS = 10Ω, CFS is calculated according to the higher value of the pole and the RHP zero of
# the system (shown as a maximizing function):
RFS = 10 # Ω
ωP3 = max(ωP1, ωZ1) * RFS
CFS = 1/(RFS*ωP3)
print("CFS =", CFS, "Farads")

CFS = 8.571428571428575e-09 Farads


In [12]:
# Input Capacitance
# Set the nominal input voltage ripple (ΔVIN_PP) by solving for the required capacitance (CIN):
# Boost:
CIN = ΔiL_PP / (8*ΔVIN_PP*fSW)
print("CIN =", CIN, "Farads")
print("Use at least 200% of this value, CIN >=", CIN*2, "Farads")
# Use DMAX to set the worst case input voltage ripple, when solving for CIN in a buck-boost regulator and DMID = 0.5
# when solving for CIN in a buck regulator.
# The minimum allowable RMS input current rating (ICIN-RMS) can be approximated:
# Boost:
ICIN_RMS = ΔiL_PP/math.sqrt(12)
print("ICIN_RMS =", ICIN_RMS, "Amps")

CIN = 1.75e-06 Farads
Use at least 200% of this value, CIN >= 3.5e-06 Farads
ICIN_RMS = 0.20207259421636903 Amps


In [13]:
# NFET
# The NFET voltage rating should be at least 15% higher than the maximum NFET drain-to-source voltage (VTMAX):
# Boost:
VTMAX = VO

# The current rating should be at least 10% higher than the maximum average NFET current (IT-MAX):
# Boost and Buck-Boost:
IT_MAX = DMAX/(1-DMAX)*ILED * 1.1
print("IT_MAX >=", IT_MAX, "Amps")
# Approximate the nominal RMS transistor current (IT-RMS) :
# Boost and Buck-Boost:
IT_RMS = ILED/Dprime*math.sqrt(D)
print("IT_RMS =", IT_RMS, "Amps")
# Given an NFET with on-resistance (RDS-ON), solve for the nominal power dissipation (PT):
RDS_ON = 1 # Ohm
PT = IT_RMS**2 * RDS_ON
print("PT =", PT, "Watts")


IT_MAX >= 3.3000000000000003 Amps
IT_RMS = 1.936491673103709 Amps
PT = 3.7500000000000018 Watts


In [14]:
# Diode
# The Schottky diode voltage rating should be at least 15% higher than the maximum blocking voltage (VRD-MAX):
# Boost:
VRD_MAX = VO * 1.15
print("VRD_MAX >=", VRD_MAX, "Volts")
# The current rating should be at least 10% higher than the maximum average diode current (ID-MAX):
# Boost and Buck-Boost:
ID_MAX = ILED * 1.1
print("ID_MAX >=", ID_MAX, "Volts")
# Replace DMAX with D in the ID-MAX equation to solve for the average diode current (ID). Given a diode with forward
# voltage (VFD), solve for the nominal power dissipation (PD):
VFD = 1.2 # Volts
PD = ID_MAX * VFD
print("PD >=", PD, "Watts")

VRD_MAX >= 41.4 Volts
ID_MAX >= 1.1 Volts
PD >= 1.32 Watts


In [15]:
# Output OVLO
# For boost and buck-boost regulators, output OVLO is programmed with the turn-off threshold voltage (VTURN-OFF)
# and the desired hysteresis (VHYSO). To set VHYSO, solve for ROV2:
ROV2 = VHYSO/20e-6
# To set VTURN-OFF, solve for ROV1:
# Boost:
ROV1 = 1.24 * ROV2 / (VTURN_OFF - 1.24)
print("ROV1 =", ROV1, "Ohms")
print("round to 60.4 or 61.2 kOhms")
print("ROV2 =", ROV2, "Ohms")
print("round to 189 or 191 kOhms")
# A small filter capacitor (COVP = 47 pF) should be added from the OVP pin to ground to reduce coupled switching
# noise.
COV = 47e-12
print("COV =", COV, "Farads")

ROV1 = 60784.31372549019 Ohms
round to 60.4 or 61.2 kOhms
ROV2 = 1899999.9999999998 Ohms
round to 189 or 191 kOhms
COV = 4.7e-11 Farads


In [17]:
# Input Undervoltage Lockout (UVLO)
# Input UVLO characteristics: VTURN_ON, VHYS
# For all topologies, input UVLO is programmed with the turn-on threshold voltage (VTURN-ON) and the desired
# hysteresis (VHYS).
# Method 2: If PWM dimming is required, a three resistor network is suggested. To set VTURN-ON, assume RUV2 =
# 10 kΩ and solve for RUV1 as in Method 1. To set VHYS, solve for RUVH:
RUV2 = 10000
RUV1 = 1.24*RUV2/(VTURN_ON-1.24)
RUVH = RUV1*(VHYS - 20e-6 * RUV2) / (20e-6 * (RUV1 + RUV2))
print("RUV1 =", RUV1, "Ohms")
print("RUV2 =", RUV2, "Ohms")
print("RUVH =", RUVH, "Ohms")

RUV1 = 1339.0928725701945 Ohms
RUV2 = 10000 Ohms
RUVH = 16533.333333333332 Ohms
